# Naive portfolio strategies - comparisons

This script presents a comparisons between the performances of "naive" portfolios for the same underlining components and rebalancing periods. 

The following "naive" portfolios are considered:

- equal weighted (all weights are equal to 1/N, where N is the number of portfolio components)
- inverse volatility (the weights are proportional with the inverse of the asset volatilities)
- inverse variance (the weights are proportional with the inverse of the asset variances)
- inverse drawdown (the weights are proportional with the asset maximum historical drawdwon observed during a predefined length historical lookback period)

We start by importing the **azapy** and other useful packages.

In [10]:
import pandas as pd

import sys
sys.path.append("..")
import azapy as az

### Collect historical market data

Note the flag `force=False`. In this case the function will attempt to read the market data from `mktdir`. If it fails then it will access the data provider servers, in this case *alphavantage*.

In [11]:
symb = ['GLD', 'TLT', 'XLV', 'VGT', 'PSJ']
        
sdate = pd.to_datetime("2012-01-01")
edate = pd.to_datetime("today")

mktdir = "./MkTdata"

mktdata = az.readMkT(symb, dstart = sdate, dend = edate, dir=mktdir, force=False) 

Read GLD form ./MkTdata
Read TLT form ./MkTdata
Read XLV form ./MkTdata
Read VGT form ./MkTdata
Read PSJ form ./MkTdata


### Set the model parameters

Since we are interested to run several portfolio strategies, it is convenient to setup the model parameters in a dictionary. 
The keys are given by the portfolio names while the model parameters are encapsulated in a dictionary:

- `type` is the name of the portfolio class 
- `m_param` is are the parameters to be set by `set_model` method

The `hlength`is set to 3.5 years. It is relevant to compute the historical volatilities, variances and maximum drawdowns.
For `P_N` the `ww` is set to None, indicating the default setup for equally weighted portfolio. 

*For more details see the documentations for Port_InvVol, Port_InvVar, Port_InvDD and Port_ConstW classes*

In [12]:
models = {'P_InvVol': {'type': 'Port_InvVol', 'm_param': {'hlength': 3.25}},
          'P_InvVaR': {'type': 'Port_InvVar', 'm_param': {'hlength': 3.25}},
          'P_InvDD': {'type': 'Port_InvDD', 'm_param': {'hlength': 3.25}},
          'P_N': {'type': 'Port_ConstW', 'm_param': {'ww': None}}}

### main computational loop

- `port` is the list of portfolio time-series
- `pp` is a list of portfolio objects. They could be interrogated later for specific information.

In [13]:
port = []
pp = {}
for key, val in models.items():
    ppz = getattr(az, val['type'])
    pp_ = ppz(mktdata, pname=key)
    pp[key] = pp_
    port_ = pp_.set_model(**val['m_param'])
    port.append(port_)

### Setup the comparisons

To facilitate a easy visual and numerical comparisons of these portfolio performances, it is very convenient to setup a `Port_Simple` class using the list of previously computed time-series. We will be interested in the analyzing the portfolio components and not the aggregated portfolio of portfolios time-series. 

Note the call to the `set_model` method that is a must.

In [14]:
ps = az.Port_Simple(port, col='close', pname='ALL')
_ = ps.set_model()

### Visualize the portfolio time-series

Note the use of the flags
- `componly=True` to retain only the portfolio components time-series and to exclude the aggregated portfolio of portfolios time-series,
- `fancy=True` to use the `plotly` interactive time series package.

*For more details see the documentation*

In [15]:
_ = ps.port_view_all(componly=True, fancy=True)

### Portfolios performances 

We use the same flags as above.

In this case the `P_InvDD` is the best performer. It also has the highest beta with higher rate of returns and smaller maximum drawdown than the equally weighted portfolio. 

In [16]:
ps.port_perf(componly=True, fancy=True)

,RR,DD,Beta,DD_date,DD_start,DD_end
symbol,,,,,,
P_InvDD,15.44,-17.09,0.903418,2020-03-18,2020-02-19,2020-04-27
P_InvVaR,12.15,-14.76,0.823112,2020-03-18,2020-02-19,2020-04-16
P_InvVol,13.04,-16.03,0.813864,2020-03-18,2020-02-19,2020-04-16
P_N,13.96,-17.44,0.800499,2020-03-18,2020-02-19,2020-04-24


### Portfolio annual returns

We used the flags
- `withcom=True` to include the components in the report
- `componly=True` to exclude the aggregated portfolio 
- `fancy=True` to report the rates in percent

The P_InvPP outperforms P_N almost every year. The exception is 2017 where the difference in the rate of returns is 25bps (bases points).

In [17]:
ps.port_annual_returns(withcomp=True, componly=True, fancy=True)

symbol,P_InvDD,P_InvVaR,P_InvVol,P_N
year,,,,
2015,-1.24%,-2.02%,-1.81%,-1.71%
2016,8.83%,6.80%,7.55%,8.33%
2017,22.71%,23.64%,23.20%,22.96%
2018,7.17%,2.63%,2.70%,2.65%
2019,28.55%,24.44%,25.21%,25.78%
2020,25.70%,18.84%,21.68%,25.55%
2021,5.50%,1.68%,3.06%,3.99%


### The historical weights for P_InvDD

Remark: At present the GLD weight is very small (less than 2%).

In [18]:
pp['P_InvDD'].get_weights(fancy=True)

,Droll,Dfix,Dhist,GLD,PSJ,TLT,VGT,XLV
0,2015-06-25,2015-06-24,2012-03-23,14.24,8.17,19.64,22.12,35.83
1,2015-09-25,2015-09-24,2012-06-22,14.60,24.76,13.46,20.07,27.11
2,2015-12-28,2015-12-24,2012-09-24,13.44,28.01,4.61,16.19,37.75
3,2016-03-28,2016-03-24,2012-12-24,10.26,20.68,16.15,24.09,28.83
4,2016-06-27,2016-06-24,2013-03-22,11.61,23.39,6.35,27.25,31.39
5,2016-09-27,2016-09-26,2013-06-26,20.90,20.94,5.68,24.39,28.10
6,2016-12-27,2016-12-23,2013-09-23,18.66,17.11,21.35,19.93,22.96
7,2017-03-28,2017-03-27,2013-12-27,15.26,18.20,22.72,19.39,24.43
8,2017-06-27,2017-06-26,2014-03-26,9.83,19.37,24.17,20.64,25.99
9,2017-09-26,2017-09-25,2014-06-25,18.38,17.66,22.04,18.24,23.69
